In [1]:
import numpy as np
import torch
class BloatWareConvTranspose2D():
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, dtype=None):
        
        if isinstance(in_channels, int) and in_channels > 0:
            self.in_channels = in_channels
        else:
            raise ValueError("Invalid in_channels")
        
        if isinstance(out_channels, int) and out_channels > 0:
            self.out_channels = out_channels
        else:
            raise ValueError("Invalid out_channels")
        
        if isinstance(groups, int) and groups > 0:
            self.groups = groups
        else:
            raise ValueError("Invalid groups")

        if isinstance(stride, int) and stride > 0:
            self.stride = stride
        else:
            raise ValueError("Invalid stride")
        
        if isinstance(padding, int) and padding > -1:
            self.padding = padding
        else:
            raise ValueError("Invalid padding")
            
        if isinstance(output_padding, int) and output_padding > -1:
            self.output_padding = output_padding
        else:
            raise ValueError("Invalid output_padding")
            
        if isinstance(dilation, int) and dilation > 0:
            self.dilation = dilation
        else:
            raise ValueError("Invalid dilation")
            
        if not((self.in_channels % self.groups == 0) and (self.out_channels % self.groups == 0)):
            raise ValueError("in_channels and out_channels must both be divisible by groups")
        
        if (self.output_padding >= self.dilation and self.output_padding >= self.stride) or (self.output_padding >= self.dilation and self.output_padding >= self.stride):
            raise ValueError("output_padding should be smaller than dilation or stride")

        if bias == True:
          self.bias = torch.rand(self.out_channels)
        else:
          self.bias = torch.zeros(self.out_channels)
    
        if isinstance(kernel_size, int):
            self.weight = torch.rand(
                self.in_channels,
                self.out_channels,
                kernel_size,
                kernel_size,
            )
            self.kernel_size = kernel_size
        else:
            raise ValueError("kernel size must be int or tuple")

        self.dtype = dtype
    
    def forward(self, input_tensor):
        result = []

        for l in range(self.out_channels):
    
          feature_map = torch.zeros((input_tensor.shape[1]-1)*self.stride + self.dilation * (self.kernel_size-1)+1, (input_tensor.shape[2]-1)*self.stride  + self.dilation * (self.kernel_size-1)+1 ) #генерация пустой feature-map
          for c in range (self.in_channels):
    
            for i in range (0, input_tensor.shape[1]):  #проход по всем пикселям изображения
              for j in range (0, input_tensor.shape[2]):
    
                val = input_tensor[c][i][j]
                proizv = val*self.weight[c][l]
    
                zero_tensor = torch.zeros((self.weight.shape[2]-1)*self.dilation+1, (self.weight.shape[3]-1)*self.dilation+1)
    
                for a in range (0, zero_tensor.shape[0], self.dilation):
                  for b in range (0, zero_tensor.shape[1], self.dilation):
                    zero_tensor[a][b] = proizv[a//self.dilation][b//self.dilation]
    
                res = np.add((zero_tensor), feature_map[i*self.stride:i*self.stride+(self.weight.shape[2]-1)*self.dilation+1, j*self.stride:j*self.stride+(self.weight.shape[3]-1)*self.dilation+1])
                feature_map[i*self.stride:i*self.stride+(self.weight.shape[2]-1)*self.dilation+1, j*self.stride:j*self.stride+(self.weight.shape[3]-1)*self.dilation+1] = res
    
    
          result.append(np.add(feature_map, np.full((feature_map.shape), self.bias[l])))
    
    
        for t in range(len(result)):
          if self.output_padding > 0:
            pad_func = torch.nn.ConstantPad1d((0, self.output_padding, 0, self.output_padding), 0)
            result[t] = pad_func(result[t])
    
          result[t] = result[t][0+self.padding:result[t].shape[0]-self.padding, 0+self.padding:result[t].shape[1]-self.padding]

        return np.array(result)

In [2]:
torchConvTranspose2D = torch.nn.ConvTranspose2d(15, 5, 3, stride=1, padding=1, dilation=1)
input_image = torch.randn(15, 50, 50)
output = torchConvTranspose2D(input_image)

In [3]:
myConvTranspose2D = BloatWareConvTranspose2D(15, 5, 3, stride=1, padding=1, dilation=1)

In [4]:
myConvTranspose2D.weight = torchConvTranspose2D.weight.detach().numpy()
myConvTranspose2D.bias = torchConvTranspose2D.bias.detach().numpy()

In [5]:
torchConvTranspose2D.weight.shape[0]

15

In [6]:
output_mock = myConvTranspose2D.forward(input_image)

In [7]:
output_mock

array([[[-0.32806575,  0.19140458, -0.38019127, ...,  0.03490135,
          0.49552664,  0.62894523],
        [ 0.29940262,  0.04433158,  0.45483455, ...,  0.289754  ,
         -0.11334548, -0.3289992 ],
        [-1.5907145 , -2.0165308 ,  0.7746327 , ..., -0.73511493,
         -0.03588326, -1.4044667 ],
        ...,
        [ 0.34540305,  0.16014487,  0.38106743, ..., -0.2564003 ,
         -0.389469  , -1.100019  ],
        [ 0.2312895 ,  0.93495333,  0.2270971 , ..., -0.47608882,
          0.66813123, -0.1007289 ],
        [-0.20820044,  0.17850864,  0.3281487 , ..., -0.32019198,
          0.7752029 , -1.0750511 ]],

       [[-0.16106942,  0.87301356, -0.98776275, ...,  0.96094114,
          0.7396459 , -0.97358143],
        [-0.62295705, -0.90292037, -1.30724   , ...,  1.459812  ,
         -1.898841  , -1.3842762 ],
        [ 1.5632981 ,  1.152181  ,  1.3168902 , ...,  0.373163  ,
         -1.5213273 ,  0.62232584],
        ...,
        [-0.5605943 , -0.3837295 , -0.9478675 , ...,  

In [8]:
output

tensor([[[-0.3281,  0.1914, -0.3802,  ...,  0.0349,  0.4955,  0.6289],
         [ 0.2994,  0.0443,  0.4548,  ...,  0.2898, -0.1133, -0.3290],
         [-1.5907, -2.0165,  0.7746,  ..., -0.7351, -0.0359, -1.4045],
         ...,
         [ 0.3454,  0.1601,  0.3811,  ..., -0.2564, -0.3895, -1.1000],
         [ 0.2313,  0.9350,  0.2271,  ..., -0.4761,  0.6681, -0.1007],
         [-0.2082,  0.1785,  0.3281,  ..., -0.3202,  0.7752, -1.0751]],

        [[-0.1611,  0.8730, -0.9878,  ...,  0.9609,  0.7396, -0.9736],
         [-0.6230, -0.9029, -1.3072,  ...,  1.4598, -1.8988, -1.3843],
         [ 1.5633,  1.1522,  1.3169,  ...,  0.3732, -1.5213,  0.6223],
         ...,
         [-0.5606, -0.3837, -0.9479,  ...,  1.3188, -0.7963, -1.7036],
         [-0.6500, -0.8411, -1.3018,  ..., -2.0380, -1.2880,  1.0633],
         [-0.1356, -0.0496,  1.6878,  ..., -0.0667, -0.9324,  0.8024]],

        [[-0.7641, -0.0114, -0.9982,  ...,  1.4993,  0.8774,  0.7354],
         [ 0.0569,  0.8396, -0.0547,  ..., -1

In [9]:
result_test = output.detach().numpy().astype("float16") == output_mock.astype(
    "float16"
)

In [10]:
result_test

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr

In [20]:
def myTranspConv2d(matrix, in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, padding_mode='zeros'):
    stride = 1

    #добавление отступов и padding в входной матрице
    pad = kernel_size - 1
    result_matrix = []
    for matr in matrix:
      zero_tensor = np.zeros((((matr.shape[0]-1)*(transp_stride)+1), ((matr.shape[1]-1)*(transp_stride)+1)))
      for a in range (0, zero_tensor.shape[0], transp_stride):
        for b in range (0, zero_tensor.shape[1], transp_stride):
          zero_tensor[a][b] = matr[a//(transp_stride)][b//(transp_stride)]

      pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')
      result_matrix.append(pad_matr)
    matrix = torch.tensor(result_matrix)

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

    #инвертирование ядра для ConvTranspose2d
    filter_for_transpose = []
    for j in range(out_channels):
      filter_in = []
      for i in range(in_channels):
        filter_in.append(np.flip(np.array(filter[j][i])))
      filter_for_transpose.append(filter_in)

    filter_for_transpose = torch.tensor(filter_for_transpose)
    filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)



    result = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels):
            val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias
      result.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))


    return np.array(result), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_val))

In [23]:
import numpy as np
import torch
import sys
import os

tensor1 = torch.rand(3, 5, 6)

result, kernel, bias_val = myTranspConv2d(tensor1, in_channels=3, out_channels=1, kernel_size=3, transp_stride=2, bias=True,)
torchFunction = torch.nn.ConvTranspose2d(in_channels=3, out_channels=1, kernel_size=3, stride=2, bias=True,)
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))


In [24]:
torchFunction(tensor1).detach().numpy().astype("float16") == result.astype(
    "float16"
)

array([[[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [ 